In [2]:
!git clone https://github.com/phucnt19/Books_and_Movies_Crawler.git

Cloning into 'Books_and_Movies_Crawler'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 55 (delta 23), reused 41 (delta 13), pack-reused 0
Unpacking objects: 100% (55/55), done.


In [28]:
!git commit . -m 'test'

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [29]:
!git push origin

The authenticity of host 'github.com (140.82.114.3)' can't be established.
ECDSA key fingerprint is SHA256:p2QAMXNIC1TJYWeIOttrVc98/R1BUFWu3/LiyKgUfQM.
Are you sure you want to continue connecting (yes/no)? ^C


In [24]:
!git remote set-url origin git@github.com:phucnt19/Books_and_Movies_Crawler.git

## Import data

In [ ]:
import pandas as pd
def read_target(file_path):
    train_target = pd.read_csv(file_path)
    train_target = train_target.sort_values(by=['fname']) #sorted by filename to match with image reading
    #train_target = train_target.fillna(method='ffill') #fill null values
    return train_target
train_target = read_target('../input/zaloaichallenge2021-5k/train_meta.csv')

In [ ]:
train_target = train_target.drop(['mask','5k'],axis=1)
train_target

In [ ]:
import numpy as np
a= sorted(train_target[train_target['distancing'].isna()].index.tolist())
a[:318]

In [ ]:
import numpy as np
import os
import cv2
def read_img(folder_path):
    train_data = []
    for f in os.listdir(folder_path):
        path = os.path.join(folder_path,f)
        image = cv2.imread(path)
        image = cv2.resize(image, (227,227))
        train_data.append(image)
    train_data = np.array(train_data)
    return train_data
train_data = read_img('../input/zaloaichallenge2021-5k/images')

In [ ]:
X = train_data
Y = np.array(train_target['5k'].values)

## Split the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [ ]:
from tensorflow.keras.utils import to_categorical
#Y_train = to_categorical(Y_train, num_classes = 2)
#Y_test = to_categorical(Y_test, num_classes = 2)

## Build model

In [ ]:
#VGG-16 pre-trained

from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model, Input

vgg = VGG16(include_top=False, input_shape=(227,227,3))
#Dong bang tat ca cac tham so
for layer in vgg.layers:
    layer.trainable=False
    #Khong lam anh huong cac tham so
    
#Them lop output
flat = Flatten()(vgg.layers[-1].output)
output = Dense(1, activation='sigmoid')(flat)

model = Model(inputs=vgg.inputs,outputs=output)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train,batch_size=64,epochs=10, validation_split=0.1)

In [ ]:
from sklearn.metrics import f1_score
y_pred_total = []
for i in model.predict(X_test):
    if i > 0.5:
        y_pred_total.append(1)
    else:
        y_pred_total.append(0)
y_pred_total = np.array(y_pred_total)
f1_score(Y_test,y_pred_total)

## Testing

In [ ]:
X_public = read_img('../input/zalopublic-test/images')

In [ ]:
Y_public = read_target('../input/zalopublic-test/public_test_meta.csv')

In [ ]:
Y_pred = model.predict(X_public)
y_pred_total = []
for i in model.predict(X_public):
    if i > 0.5:
        y_pred_total.append(1)
    else:
        y_pred_total.append(0)
Y_public['5k'] = y_pred_total
Y_public = Y_public.sort_values(by=['image_id'])
Y_public.to_csv('output.csv')
Y_public

In [ ]:
Y_public.loc[Y_public['5k'] == 0].count()